In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
train = pd.read_csv("data/cleaned_train.csv")
test = pd.read_csv("data/test.csv")
pd.set_option('display.max_columns', None)
train

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,complete_resp_total
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.061430,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0,2.0,55.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.211720,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,453.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,0.0,28.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,577.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.220330,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0,1.0,44.0
4,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,NaN,60.0,73.0,102.0,NaN,NaN,NaN,Summer,12.0,0.0,16.5,2.0,17.9,2.0,6.0,0.0,10.0,1.0,11.0,1.0,8.0,0.0,Summer,2.0,4.33036,30.1865,1330.970,1996.45,30.21240,84.0285,16.6877,13.498800,67.97150,2.0,32.9141,20.90200,79.6982,35.3804,63.1265,NaN,NaN,Spring,4.110,Summer,40.0,56.0,Spring,0.0,1.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,ff6c2bb8,Fall,8,0,NaN,NaN,Fall,17.139810,52.5,67.2,25.0,60.0,65.0,112.0,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,8.0,1.0,10.0,1.0,12.0,1.0,Fall,3.0,3.20303,17.1417,1035.270,1759.96,11.00630,52.5331,13.4004,3.741300,14.66690,1.0,25.7118,15.81500,49.3301,20.2645,36.7181,NaN,NaN,Fall,3.440,Fall,41.0,58.0,Fall,2.0,0.0,22.0
2667,ff759544,Summer,7,1,NaN,NaN,Summer,13.927006,48.5,46.6,23.0,65.0,75.0,105.0,NaN,NaN,NaN,Summer,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,9.0,0.0,8.5,0.0,4.5,0.0,Fall,1.0,2.36680,13.6457,966.287,1256.17,9.98802,45.1853,13.2315,0.414263,1.41470,1.0,20.0572,15.14000,42.8185,18.0937,30.0453,NaN,NaN,NaN,NaN,Summer,48.0,67.0,Summer,0.0,1.0,33.0
2668,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,NaN,71.0,70.0,104.0,NaN,NaN,NaN,Fall,16.0,0.0,18.0,1.0,19.9,2.0,10.0,1.0,8.0,1.0,9.0,1.0,12.0,1.0,Fall,3.0,4.52277,16.3642,1206.880,2051.70,19.46110,70.8117,14.0629,2.301380,11.58830,1.0,33.3709,17.97970,66.2889,29.7790,52.8320,NaN,NaN,Winter,3.260,Winter,35.0,50.0,Fall,1.0,1.0,32.0
2669,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,NaN,79.0,99.0,116.0,NaN,NaN,NaN,Winter,15.0,1.0,18.5,2.0,15.8,2.0,0.0,0.0,10.0,1.0,10.0,1.0,

In [3]:
train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,complete_resp_total
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0,2.0,55.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,453.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0,0.0,28.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,577.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0,1.0,44.0
4,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,NaN,60.0,73.0,102.0,NaN,NaN,NaN,Summer,12.0,0.0,16.5,2.0,17.9,2.0,6.0,0.0,10.0,1.0,11.0,1.0,8.0,0.0,Summer,2.0,4.33036,30.1865,1330.970,1996.45,30.21240,84.0285,16.6877,13.49880,67.97150,2.0,32.9141,20.90200,79.6982,35.3804,63.1265,NaN,NaN,Spring,4.110,Summer,40.0,56.0,Spring,0.0,1.0,34.0


In [4]:
test.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
test['is_test'] = True  # Add a flag to identify test data
train['is_test'] = False
combined_data = pd.concat([train, test], ignore_index=True)

In [6]:
# Split back into train and test datasets
train_data_preprocessed = combined_data[combined_data['is_test'] == False].drop(columns=['is_test'])
test_data_preprocessed = combined_data[combined_data['is_test'] == True].drop(columns=['is_test'])
train_data_preprocessed = train_data_preprocessed.drop(columns="id")
# Print the results
print("Preprocessed Train Data:")
print(train_data_preprocessed)

print("\nPreprocessed Test Data:")
print(test_data_preprocessed)

Preprocessed Train Data:
     Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex CGAS-Season  \
0                         Fall                5                0      Winter   
1                       Summer                9                0         NaN   
2                       Summer               10                1        Fall   
3                       Winter                9                0        Fall   
4                       Spring               13                1      Winter   
...                        ...              ...              ...         ...   
2666                      Fall                8                0         NaN   
2667                    Summer                7                1         NaN   
2668                      Fall               13                0      Spring   
2669                      Fall               11                0      Spring   
2670                    Spring               13                0      Spring   

      CGAS-CGA

In [10]:
X = train_data_preprocessed.drop(columns=['sii', 'complete_resp_total'])
y = train_data_preprocessed['sii']

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
'''
imputer = KNNImputer(n_neighbors=4)
X_imputed = imputer.fit_transform(X)
# Drop the missing column
X = X.drop(columns=['Fitness_Endurance-Time_Mins'])
# Convert X_imputed back to a DataFrame with matching columns
X = pd.DataFrame(X_imputed, columns=X.columns)
'''
categorical_features = [
    'Basic_Demos-Sex',
    'FGC-FGC_CU_Zone',
    'FGC-FGC_GSND_Zone',
    'FGC-FGC_GSD_Zone',
    'FGC-FGC_PU_Zone',
    'FGC-FGC_SRL_Zone',
    'FGC-FGC_SRR_Zone',
    'FGC-FGC_TL_Zone',
    'BIA-BIA_Activity_Level_num',
    'BIA-BIA_Frame_num',
    'PreInt_EduHx-computerinternet_hoursday'
]

numerical_features = X.columns.difference(categorical_features)  # All other columns

# Create pipelines for numerical and categorical features
numerical_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=4))
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing categorical values with the most frequent value
    ('encoder', OrdinalEncoder())  # Encode categories into numbers
])

# Combine into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

# Apply the preprocessor
X_processed = preprocessor.fit_transform(X)

# Convert the preprocessed data back to a DataFrame
X = pd.DataFrame(X_processed, columns=numerical_features.tolist() + categorical_features)

print("Preprocessing completed successfully!")

# Oversample the minority class using ADASYN
adasyn = ADASYN()
X_ADASYN, y_ADASYN = adasyn.fit_resample(X, y)

# Check class distribution after resampling
print("Class distribution after ADASYN:", pd.Series(y_ADASYN).value_counts())
X_train, X_val, y_train, y_val = train_test_split(X_ADASYN, y_ADASYN, test_size=0.2, random_state=42)

ValueError: could not convert string to float: 'Fall'

In [9]:
print(y_train.value_counts())

complete_resp_total
0.0     238
20.0     55
30.0     50
23.0     45
25.0     45
       ... 
76.0      1
82.0      1
90.0      1
91.0      1
93.0      1
Name: count, Length: 93, dtype: int64


In [15]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(
    iterations=1000,       # Number of boosting iterations
    learning_rate=0.01,    # Step size for weight updates
    depth=8,              # Depth of the trees
    loss_function='RMSE', # Regression problem
    verbose=200,
    cat_features=categorical_features,
    l2_leaf_reg=5)

# Define scoring metric (neg_mean_squared_error for regression)
scoring = 'neg_mean_squared_error'

# Perform Cross-Validation
scores = cross_val_score(model, X, y, cv=5, scoring=scoring)

# Convert to positive MSE
mse_scores = -scores

# Calculate RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

# Print Results
print("Mean RMSE:", mean_rmse)
print("RMSE Scores for each fold:", rmse_scores_per_fold)

0:	learn: 20.3763306	total: 162ms	remaining: 2m 41s
200:	learn: 16.3053689	total: 25.5s	remaining: 1m 41s
400:	learn: 15.0704493	total: 52.9s	remaining: 1m 18s
600:	learn: 14.3382443	total: 1m 19s	remaining: 52.5s
800:	learn: 13.6876072	total: 1m 45s	remaining: 26.3s
999:	learn: 13.2791275	total: 2m 11s	remaining: 0us
0:	learn: 20.2888132	total: 135ms	remaining: 2m 14s
200:	learn: 16.2782862	total: 27.3s	remaining: 1m 48s
400:	learn: 15.0622080	total: 56.1s	remaining: 1m 23s
600:	learn: 14.3444473	total: 1m 20s	remaining: 53.5s
800:	learn: 13.7338037	total: 1m 48s	remaining: 26.8s
999:	learn: 13.1956950	total: 2m 16s	remaining: 0us
0:	learn: 20.2624518	total: 119ms	remaining: 1m 59s
200:	learn: 16.2488939	total: 27s	remaining: 1m 47s
400:	learn: 15.0606484	total: 54.7s	remaining: 1m 21s
600:	learn: 14.3828413	total: 1m 21s	remaining: 54.3s
800:	learn: 13.8016856	total: 1m 48s	remaining: 27.1s
999:	learn: 13.4295693	total: 2m 14s	remaining: 0us
0:	learn: 19.9212941	total: 126ms	remainin

In [3]:
# Identify columns to encode (all that end with '-Season')
columns_to_encode = [col for col in train.columns if col.endswith('Season')]

# Perform one-hot encoding
train = pd.get_dummies(train, columns=columns_to_encode, prefix=columns_to_encode)
test = pd.get_dummies(test, columns=columns_to_encode, prefix=columns_to_encode)

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Create a OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', ohe, categorical_features),
    ],
    remainder='passthrough'  # This will keep numerical columns as they are if not scaled.
)

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

rf_model = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('rf', RandomForestRegressor(
        n_estimators=50,    # Number of trees
        max_depth=5,         # Similar depth to CatBoost example
        min_samples_split=2,
        min_samples_leaf=2,
        random_state=42      # For reproducibility
    ))
])

# Define scoring metric (neg_mean_squared_error for regression)
scoring = 'neg_mean_squared_error'

# Perform Cross-Validation
scores = cross_val_score(rf_model, X, y, cv=5, scoring=scoring)

# Convert to positive MSE
mse_scores = -scores

# Calculate RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

# Print Results
print("Mean RMSE:", mean_rmse)
print("RMSE Scores for each fold:", rmse_scores_per_fold)

Mean RMSE: 17.221198882932832
RMSE Scores for each fold: [17.2224965  16.85024082 16.60119615 17.96469116 17.43483902]


In [11]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the parameter grid for RandomForestRegressor
param_grid = {
    'rf__n_estimators': [50, 100, 200],      # Different numbers of trees
    'rf__max_depth': [5, 8, 10],             # Different tree depths
    'rf__min_samples_split': [2, 5, 10],     # Minimum samples to split a node
    'rf__min_samples_leaf': [1, 2, 4]        # Minimum samples per leaf
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_model,    # The pipeline object
    param_grid=param_grid,
    cv=5,                  # 5-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,             # Use all available cores for faster training
    verbose=3              # Print detailed logs
)

# Fit the grid search on your data
grid_search.fit(X, y)

# View the best parameters found by GridSearchCV
print("Best parameters found:", grid_search.best_params_)

# The best estimator after tuning
best_model = grid_search.best_estimator_

# Evaluate the best model using the validation scores
best_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')
best_mse_scores = -best_scores
mean_rmse = np.sqrt(np.mean(best_mse_scores))
rmse_scores_per_fold = np.sqrt(best_mse_scores)

print("Mean RMSE after tuning:", mean_rmse)
print("RMSE Scores for each fold after tuning:", rmse_scores_per_fold)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters found: {'rf__max_depth': 5, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 50}
Mean RMSE after tuning: 17.221198882932832
RMSE Scores for each fold after tuning: [17.2224965  16.85024082 16.60119615 17.96469116 17.43483902]


In [31]:
'''
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'iterations': [200],
    'learning_rate': [0.1],
    'depth': [6, 8],
    'l2_leaf_reg': [1, 3, 5]  # Regularization parameter
}

# Initialize model
model = CatBoostRegressor(loss_function='RMSE', cat_features=categorical_features, verbose=0)

# Use GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',
    verbose=3
)

# Perform grid search
grid_search.fit(X, y)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", (-grid_search.best_score_) ** 0.5)
'''

'\nfrom sklearn.model_selection import GridSearchCV\n\n# Define parameter grid\nparam_grid = {\n    \'iterations\': [200],\n    \'learning_rate\': [0.1],\n    \'depth\': [6, 8],\n    \'l2_leaf_reg\': [1, 3, 5]  # Regularization parameter\n}\n\n# Initialize model\nmodel = CatBoostRegressor(loss_function=\'RMSE\', cat_features=categorical_features, verbose=0)\n\n# Use GridSearchCV\ngrid_search = GridSearchCV(\n    estimator=model,\n    param_grid=param_grid,\n    cv=3,  # 3-fold cross-validation\n    scoring=\'neg_mean_squared_error\',\n    verbose=3\n)\n\n# Perform grid search\ngrid_search.fit(X, y)\n\n# Best parameters and score\nprint("Best Parameters:", grid_search.best_params_)\nprint("Best RMSE:", (-grid_search.best_score_) ** 0.5)\n'

In [16]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# You already have:
# model: a CatBoostRegressor instance with specified parameters (not yet fitted)
# rf_model: a pipeline that encodes categorical features and applies RandomForestRegressor

# Create an ensemble of the CatBoost model and the Random Forest model
ensemble = VotingRegressor(estimators=[
    ('cat', model),
    ('rf', rf_model)
])

# Perform cross-validation on the ensemble
scoring = 'neg_mean_squared_error'
scores = cross_val_score(ensemble, X, y, cv=5, scoring=scoring)

# Convert from negative MSE to positive MSE
mse_scores = -scores

# Compute RMSE
mean_rmse = np.sqrt(np.mean(mse_scores))
rmse_scores_per_fold = np.sqrt(mse_scores)

print("Mean RMSE (Ensemble):", mean_rmse)
print("RMSE Scores for each fold (Ensemble):", rmse_scores_per_fold)

0:	learn: 20.3763306	total: 58.7ms	remaining: 58.6s
200:	learn: 16.3053689	total: 24.4s	remaining: 1m 36s
400:	learn: 15.0704493	total: 49.9s	remaining: 1m 14s
600:	learn: 14.3382443	total: 1m 20s	remaining: 53.3s
800:	learn: 13.6876072	total: 1m 47s	remaining: 26.8s
999:	learn: 13.2791275	total: 2m 18s	remaining: 0us
0:	learn: 20.2888132	total: 148ms	remaining: 2m 27s
200:	learn: 16.2782862	total: 30.7s	remaining: 2m 1s
400:	learn: 15.0622080	total: 1m 2s	remaining: 1m 32s
600:	learn: 14.3444473	total: 1m 33s	remaining: 1m 1s
800:	learn: 13.7338037	total: 2m 2s	remaining: 30.5s
999:	learn: 13.1956950	total: 2m 31s	remaining: 0us
0:	learn: 20.2624518	total: 98.1ms	remaining: 1m 37s
200:	learn: 16.2488939	total: 27.3s	remaining: 1m 48s
400:	learn: 15.0606484	total: 54.6s	remaining: 1m 21s
600:	learn: 14.3828413	total: 1m 21s	remaining: 54.2s
800:	learn: 13.8016856	total: 1m 47s	remaining: 26.8s
999:	learn: 13.4295693	total: 2m 11s	remaining: 0us
0:	learn: 19.9212941	total: 103ms	remaini